In [ ]:
!pip install theano numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668109 sha256=8359a81c65a56036012acc5e86c349e5f1ae32b70479b40ee87b5c62815ef633
  Stored in directory: /root/.cache/pip/wheels/d9/e6/7d/2267d21a99e4ab8276f976f293b4ff23f50c9d809f4a216ebb
Successfully built theano


In [ ]:
import numpy as np
import theano
import theano.tensor as T
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.utils import np_utils

In [ ]:
# Load and preprocess the IMDB dataset
max_features = 20000
maxlen = 100
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)

In [ ]:
# Define Theano symbolic variables
X_theano = T.imatrix('X')
y_theano = T.matrix('y')

In [ ]:
embed_dim = 128
hidden_dim = 64
W_embed = theano.shared(np.random.randn(max_features, embed_dim), name='W_embed')
W_hidden = theano.shared(np.random.randn(embed_dim, hidden_dim), name='W_hidden')
b_hidden = theano.shared(np.zeros((hidden_dim,)), name='b_hidden')
W_output = theano.shared(np.random.randn(hidden_dim, 2), name='W_output')
b_output = theano.shared(np.zeros((2,)), name='b_output')

In [ ]:
# Define the forward pass
embedded = T.dot(X_theano, W_embed)
hidden = T.tanh(T.dot(embedded, W_hidden) + b_hidden)
y_pred = T.nnet.softmax(T.dot(hidden, W_output) + b_output)

# Define the cost function (cross-entropy)
cost = T.mean(T.nnet.categorical_crossentropy(y_pred, y_theano))

# Define gradients
params = [W_embed, W_hidden, b_hidden, W_output, b_output]
grads = T.grad(cost, params)

In [ ]:
updates = [(param, param - 0.01 * grad) for param, grad in zip(params, grads)]
train = theano.function(inputs=[X_theano, y_theano], outputs=cost, updates=updates)
predict = theano.function(inputs=[X_theano], outputs=T.argmax(y_pred, axis=1))

In [ ]:
# Train the model
batch_size = 128
num_batches = X_train.shape[0] // batch_size
epochs = 5
for epoch in range(epochs):
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size
        train(X_train[start:end], y_train[start:end])

# Evaluate the model
predictions = predict(X_test)
accuracy = np.mean(predictions == np.argmax(y_test, axis=1))
print("Accuracy:", accuracy)